# Using E3 in a Jupyter Notebook

As an API, E3 is designed to be callable from any programming language capable of making API requests. This example will illustrate the process using Python and assumes you already have Python installed.

**If running locally**, the **requests** library will need to be installed to access the API. This can be done using the following command in your computer's terminal/command prompt provided you have Python already installed (if **requests** is already installed you can skip this step).

In [ ]:
%pip install -q requests

**If running Jupyter from the browser (JupyterLite)** you will need to run the following commands. Note that requests doesn't work in JupyterLite.

In [ ]:
from pyodide.ffi import to_js
from IPython.display import JSON
from js import Object, fetch

With requests installed the only items remaining are to construct the JSON string to send to the API and to properly construct the API request in python. This notebook will cover constructing the JSON string from a CSV file. For examples of how to use input from JSON, XML and XLS/XLSX files see the **E3 Example JSON**, **E3 Example XML** and **E3 Example Excel** (in development) files, respectively. 

## Constructing the JSON string

The format for the JSON request must be correct, otherwise E3 will return an error. In this section we will look at the required format for individual objects within E3 before putting all of them into a single JSON string for an API request. The discussion here will be focused on formatting, for a more in depth look please see the technical manual for E3. We'll start with the **Analysis** object format. **Note:** If you double click on some cells you may see **\<br>** and **\&emsp;** tags. These are used exclusively for formatting in Jupyter and **should not appear in the input JSON string sent to the E3 API.**

"analysisObject" : { <br>
&emsp;"type" : "BCA", <br>
&emsp;"projectType" : "Infrastructure", <br>
&emsp;"outputObjects" : [ <br>
&emsp;&emsp;"required", <br>
&emsp;&emsp;"measure", <br>
&emsp;&emsp;"optional" <br>
&emsp;], <br>
&emsp;"studyPeriod" : 50, <br>
&emsp;"timestepValue" : "Year", <br>
&emsp;"timestepComp" : "End_Of_Year", <br>
&emsp;"outputReal" : true, <br>
&emsp;"interestRate": 0.03, <br>
&emsp;"discountRateReal" : 0.03, <br>
&emsp;"inflationRate" : 0.023, <br>
&emsp;"marr" : 0.03, <br>
&emsp;"reinvestRate" : 0.03, <br>
&emsp;"federalIncomeRate": 0.22, <br>
&emsp;"otherIncomeRate": 0.05, <br>
&emsp;"numberOfAlternatives" : 2, <br>
&emsp;"baseAlternative" : 0 <br>
}

There should only be one Analysis object for the input. **Alternative** objects have the following basic format.

{ <br>
&emsp;"id" : 0, <br>
&emsp;"name" : "Alt 0 Status Quo", <br>
&emsp;"bcns" : [ <br>
&emsp;&emsp;0 <br>
&emsp;] <br>
}

Multiple alternatives are required for analysis since E3 assumes comparison to a baseline. If only one alternative exists in an analysis you can simply define an alternative with a single cost object with zero quant or quantValue (See the examples for BCNs. An example with two alternatives is presented here. Note that the **alternativeObjects** key is now prepended and the  associated value is now a list of alternative objects.

"alternativeObjects" : [ <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 0, <br>
&emsp;&emsp;"name" : "Alt 0 Status Quo", <br>
&emsp;&emsp;"bcns" : [ <br>
&emsp;&emsp;&emsp;0 <br>
&emsp;&emsp;] <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 1, <br>
&emsp;&emsp;"name" : "Alt 1 Build Monument", <br>
&emsp;&emsp;"bcns" : [ <br>
&emsp;&emsp;&emsp;1, <br>
&emsp;&emsp;&emsp;2, <br>
&emsp;&emsp;&emsp;3, <br>
&emsp;&emsp;&emsp;4 <br>
&emsp;&emsp;] <br>
&emsp;} <br>
]

An individual **BCN** object has the following format if the BCN doesn't recur.

{ <br>
&emsp;"id" : 0, <br>
&emsp;"altIds" : [ <br>
&emsp;&emsp;0 <br>
&emsp;], <br>
&emsp;"type" : "Cost", <br>
&emsp;"subType" : "Direct", <br>
&emsp;"name" : "Investment Cost - Status Quo", <br>
&emsp;"tags" : "Initial Investment", <br>
&emsp;"initialOccurrence" : 0, <br>
&emsp;"real" : true, <br>
&emsp;"invest" : true, <br>
&emsp;"residualValue" : false, <br>
&emsp;"residualValueOnly" : false, <br>
&emsp;"quantityValue" : 0.0, <br>
&emsp;"quantity" : 1.0 <br>
}

If the **BCN** does recur it has the following general format.

{ <br>
&emsp;"id" : 2, <br>
&emsp;"altIds" : [ <br>
&emsp;&emsp;1 <br>
&emsp;], <br>
&emsp;"type" : "Cost", <br>
&emsp;"subType" : "Direct", <br>
&emsp;"name" : "Maintain Monument", <br>
&emsp;"tags" : "OMR Costs", <br>
&emsp;"initialOccurrence" : 1, <br>
&emsp;"real" : true, <br>
&emsp;"invest" : false, <br>
&emsp;"life" : 50, <br>
&emsp;"residualValue" : false, <br>
&emsp;"residualValueOnly" : false, <br>
&emsp;"recur" : { <br>
&emsp;&emsp;"interval" : 1, <br>
&emsp;&emsp;"end" : 50, <br>
&emsp;&emsp;"varRate" : "Percent_Delta", <br>
&emsp;&emsp;"varValue" : 0.0 <br>
&emsp;}, <br>
&emsp;"quantityValue" : 1000.0, <br>
&emsp;"quantity" : 1.0 <br>
}

Now putting all **BCN** objects together.

"bcnObjects" : [ <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 0, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;0 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Investment Cost - Status Quo", <br>
&emsp;&emsp;"tags" : "Initial Investment", <br>
&emsp;&emsp;"initialOccurrence" : 0, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : true, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 0.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 1, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Construct Monument", <br>
&emsp;&emsp;"tags" : "Investment Cost", <br>
&emsp;&emsp;"initialOccurrence" : 0, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : true, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 100000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 2, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Cost", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Maintain Monument", <br>
&emsp;&emsp;"tags" : "OMR Costs", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"life" : 50, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"recur" : { <br>
&emsp;&emsp;&emsp;"interval" : 1, <br>
&emsp;&emsp;&emsp;"end" : 50, <br>
&emsp;&emsp;&emsp;"varRate" : "Percent_Delta", <br>
&emsp;&emsp;&emsp;"varValue" : 0.0 <br>
&emsp;&emsp;}, <br>
&emsp;&emsp;"quantityValue" : 1000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 3, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Benefit", <br>
&emsp;&emsp;"subType" : "Direct", <br>
&emsp;&emsp;"name" : "Free Publicity", <br>
&emsp;&emsp;"tags" : "Publicity", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"quantityValue" : 50000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;}, <br>
&emsp;{ <br>
&emsp;&emsp;"id" : 4, <br>
&emsp;&emsp;"altIds" : [ <br>
&emsp;&emsp;&emsp;1 <br>
&emsp;&emsp;], <br>
&emsp;&emsp;"type" : "Benefit", <br>
&emsp;&emsp;"subType" : "Externality", <br>
&emsp;&emsp;"name" : "Increased Tourism", <br>
&emsp;&emsp;"tags" : "Tourism", <br>
&emsp;&emsp;"initialOccurrence" : 1, <br>
&emsp;&emsp;"real" : true, <br>
&emsp;&emsp;"invest" : false, <br>
&emsp;&emsp;"residualValue" : false, <br>
&emsp;&emsp;"residualValueOnly" : false, <br>
&emsp;&emsp;"recur" : { <br>
&emsp;&emsp;&emsp;"interval" : 1 <br>
&emsp;&emsp;}, <br>
&emsp;&emsp;"quantityValue" : 5000.0, <br>
&emsp;&emsp;"quantity" : 1.0 <br>
&emsp;} <br>
]

Now that the format is defined we can look at the CSV file (**Jupyter CSV.csv**). Since CSVs do not have a strict structure as compared to the required JSON formatting it's possible to construct a valid CSV for JSON construction that looks different from the one provided for this example. So long as the structure of values in the CSV file is in a form that can easily be parsed and converted to either a dictionary (we'll use this as an intermediary to constructing the JSON string) or a JSON string directly the format won't matter, though the code to parse the file will need to be altered for the given structure.

A key aspect of making the file parsable is to ensure there are patterns that allow individual objects to be easily recognizable in terms of where they start and where they stop. The easiest way would be to make the CSV format match the JSON structure almost exactly. Doing so would allow it to be brought in as a string literal requiring only minor cleanup potentially only removal of white space characters before passing it directly to E3. There are valid reasons why this might be undesirable, including readability or requirements for whatever program generated the file. Regardless we will focus on an example where the CSV structure mirrors the JSON, but not exactly.

For the example CSV major object types (**analysisObjects**, **alternativeObjects**, and **bcnObjects**) all exist in a row of the CSV file with only a curly bracket in the second row, making them easy to identify when parsing. Furthermore the alternatives and bcns within their respective larger object groupings all have the same final attribute making it easy to know when they stop. Additionally, curly brackets separate each individual object from another. These patterns are easy to generate when constructing the CSV and easy to exploit when parsing it. Also note that some patterns exist only to make visual inspection of the CSV file easier and are treated by the code identically to other patterns.

In [ ]:
def buildDictFromCSV(iFile, sub_dict = {}, on_line = []):
    line = iFile.readline()
    line = line.strip('\n')
    line = line.split(',')
    global current_line
    
    # A single curly bracket denotes that the following dictionary is part of a list. 
    if line[0] == "{":
        dict_list = []
        list_dict = buildDictFromCSV(iFile, dict(), current_line)
        dict_list.append(list_dict)
        # A closing followed by an opening curly bracket denotes the continuation of a list
        if current_line == ['}', '{']:
            in_list = True
            while in_list:
                list_dict = buildDictFromCSV(iFile, dict(), current_line)
                if current_line != ['}', '{']:
                    in_list = False
                dict_list.append(list_dict)
        # A closing curly bracket followed by a closing square bracket denotes the end of a list.
        if current_line == ["}", "]"]:
            return dict_list
        
    # A closing curly bracket in the first column denotes the end of a key object in the input.
    elif line[0] == "}":
        current_line = line
        return sub_dict
    
    elif len(line) == 2 and line[1] == "}":
        current_line = line
        return endOfKeyObject(iFile, sub_dict)

    # A square bracket in the second column denotes that the following dictionaries are in a list.
    elif line[1] == "[" or (len(on_line) == 1 and on_line[1] == "["):
        sub_dict[line[0]] = buildDictFromCSV(iFile)
        return endOfKeyObject(iFile, sub_dict)
    
    # A curly bracket next to a valid attribute name (i.e. not a bracket) denotes a sub-dictionary for an attribute
    elif len(line) == 2 and line[1] == "{" and line[0] != "}":
        sub_dict[line[0]] = buildDictFromCSV(iFile, dict())
        return endOfKeyObject(iFile, sub_dict)
        
    else:
        sub_dict[line[0]] = line[1]
        sub_dict = buildDictFromCSV(iFile, sub_dict)
        return sub_dict
    
def endOfKeyObject(iFile, sub_dict):
    try:
        buildDictFromCSV(iFile, sub_dict)
        return sub_dict
    except IndexError:
        return sub_dict

We will now call the function

In [ ]:
inputFile = "Jupyter CSV.csv" # This will work so long as the CSV file is in the same directory as the notebook
key_objects = ["analysisObject","alternativeObjects","bcnObjects"]

# We could use inputFile.open() if we want the whole file in memory, however we only need one pass to parse so using
# with.open() works just as well with the added bonus of not requiring us to explicitly close the file when done
input_dict = dict()
with open(inputFile, 'r') as iFile:
    for line in iFile:
        line = line.split(',')
        # The following if statement could be omitted if buildDictFromCSV were altered to be completely agnostic
        # to how it gets initiated. The key_objects list is for later clean up of the dictionary, the additional code
        # for this loop is minimal and the nature of E3 requires input object groupings be grouped in a particular
        # manner so the explicit nature of this step is fine here.
        if line[0] in key_objects:
            input_dict[line[0]] = buildDictFromCSV(iFile, dict())

print(input_dict)

At this point some cleanup needs to be done before it's ready for the API request. First, the lists (i.e. [1;2;3;4]) need to be converted to a python format ([1,2,3,4]) and all numbers need to be converted to int or float. Lastly, the boolean variables need to be converted to their Python equivalant values (TRUE to True, FALSE to False). The CSV structure could be used to enforce this however it's instructive to illustrate how that process can be accomplished here.

In [ ]:
def str_to_num(d):
    tol = 1e-6
    for key, value in d.items():
        if isinstance(value, dict):
            str_to_num(value)
        elif isinstance(value, list):
            for item in value:
                str_to_num(item)
        else:
            try:
                valueInt = int(value)
                valueFloat = float(value)
                if abs(float(valueInt)-valueFloat) < tol:
                    d[key] = valueInt
                else:
                    d[key] = valueFloat
            except ValueError:
                try: 
                    d[key] = float(value)
                except ValueError:
                    if value == 'FALSE':
                        d[key] = False
                        
                    elif value == 'TRUE':
                        d[key] = True

str_to_num(input_dict)
print(input_dict)

Now list variables in the dictionary need to be properly adjusted. This can be done through a check of either the placeholder symbols used to keep the CSV readable, in this case ";" or by looking for square brackets in a string variable.

In [ ]:
def list_convert(d):
    tol = 1e-6
    for key, value in d.items():
        if isinstance(value, dict):
            list_convert(value)
        elif isinstance(value, list):
            for item in value:
                list_convert(item)
        else:
            try:
                if "[" in value and isinstance(value,str):
                    value = value.replace(';',',')
                    value = value.strip('[')
                    value = value.strip(']')
                    value = value.split(',')
                    try:
                        for i in range(len(value)):
                            itemInt = int(value[i])
                            itemFloat = float(value[i])
                            if abs(float(itemInt)-itemFloat) < tol:
                                value[i] = itemInt
                            else:
                                value[i] = itemFloat
                        d[key] = value
                    except ValueError:
                        d[key] = value
            except TypeError:
                pass

list_convert(input_dict)
print(input_dict)

An API key is required to use E3. Paste yours where it says API_KEY. To get an API key you need to go to https://e3.nist.gov, create a log in and request a new key. The key should be passed in the **Authorization** header. The **Accept** header defines the output the client will accept while the **Content-Type** header defines the format of the sent data.

In [ ]:
headers = {'Content-Type': "application/json", 
           'Accept': "application/json", 
           'Authorization': "Api-Key: CXizFToEOIWa.a8606a6d-d9ae-4907-93a7-ea64c253c37f"
          }

The full request can now be made. **If running locally use the request library and the following code.**

In [ ]:
import requests
import json
api_url = "https://e3.nist.gov/api/v2/analysis"
response = requests.post(api_url, data=json.dumps(input_dict), headers=headers)
response.json()
response.status_code

# The status code can be dropped if the request is successful
if response.status_code == 200:
    response = response.json()
    print(response)
else:
    print("Error:", response.status_code)

**If running through JupyterLite use the following code.**

In [ ]:
import json

resp = await fetch("https://e3.nist.gov/api/v2/analysis",
                       method = "POST",
                       body = json.dumps(input_dict),
                       credentials = "same-origin",
                       headers = Object.fromEntries(to_js(headers))
                      )
response = await resp.text()
response = json.loads(response)

print(response)

We can now pull output from the reponse object.

In [ ]:
print("Net Benefits: Alt 0 Status Quo:", response['measure'][0]['netBenefits'])
print("Net Benefits: Alt 1 Build Monument:", response['measure'][1]['netBenefits'])
print("BCR: Alt 0 Status Quo:", response['measure'][0]['bcr'])
print("BCR: Alt 1 Build Monument:", response['measure'][1]['bcr'])

Now we can write the output to an output CSV file

In [ ]:
import csv
output_file = 'Jupyter CSV Output.csv'
output_dict = response

def writeDictToCSV(d, file):
    for key, value in d.items():
        if isinstance(value, dict):
            writeDictToCSV(value, file)
        elif isinstance(value, list):
            try:
                file.write(str(key) + "\n")
                for list_item in value:
                    writeDictToCSV(list_item, file)
            except AttributeError:
                to_write = str(value) + '\n'
        else:
            to_write = ','.join(str(i) for i in [key, value])
            to_write = to_write + '\n'
            file.write(to_write)

with open(output_file, 'w') as oFile:
    writeDictToCSV(output_dict, oFile)